In [1]:
import geopandas as gpd
import numpy as np
import sys

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Clip DEM and visible image for rayed craters (SLDEM2013)

In [2]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya.shp'
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 8.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/TCOMAP_RayedCraters/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters"

#### Let's limit SLDEM2013 to crater diameter below 7500 m in diameter

In [4]:
gdf = gpd.read_file(location_of_craters)
gdf.columns

Index(['lon', 'lat', 'tag', 'CRATER_ID', 'MED_DIAM', 'UNC_DIAM', 'DIAM_25',
       'DIAM_75', 'DIAM_MIN', 'DIAM_MAX', 'DEPTH', 'MED_H', 'UNC_H', 'H_25',
       'H_75', 'H_MIN', 'H_MAX', 'MED_MCW', 'UNC_MCW', 'MCW_25', 'MCW_75',
       'MCW_MIN', 'MCW_MAX', 'MED_UCW', 'UNC_UCW', 'UCW_25', 'UCW_75',
       'UCW_MIN', 'UCW_MAX', 'MED_CSE', 'UNC_CSE', 'CSE_25', 'CSE_75',
       'CSE_MIN', 'CSE_MAX', 'dD', 'MARE', 'QUALITY', 'TYPE_CRAT',
       'IMPACT_MLT', 'WITH_LGCRA', 'RA_CFLOOR', 'RA_EJEC', 'RAYS',
       'CEJ_TEXTUR', 'NO_CRAT', 'LIGHT_CRA', 'MED_CRA', 'HEAV_CRA', 'TOTAL',
       'CLEM', 'Shape_Leng', 'Shape_Area', 'diam', 'geometry'],
      dtype='object')

In [5]:
gdf_selection = gdf[gdf.diam < 7500.0]

In [6]:
gdf_selection.head(5)

,lon,lat,tag,CRATER_ID,MED_DIAM,UNC_DIAM,DIAM_25,DIAM_75,DIAM_MIN,DIAM_MAX,...,NO_CRAT,LIGHT_CRA,MED_CRA,HEAV_CRA,TOTAL,CLEM,Shape_Leng,Shape_Area,diam,geometry
16,-172.210501,56.619767,standard,crater0016,5488.27,35.8802,5289.83,5643.39,5008.120,10081.40,...,0.0,0.0,0.0,0.0,-3.0,1.0,25463.517125,4.531376e+07,5634.171259,POINT (-5221999.382 1716901.906)
17,-174.973257,59.415284,standard,crater0017,1204.95,46.1607,1017.46,1305.17,743.362,2648.64,...,0.0,0.0,0.0,0.0,-2.0,3.0,6604.531640,2.944041e+06,1381.046473,POINT (-5305964.776 1801820.287)
18,-162.804261,59.203522,standard,crater0018,1455.44,49.4992,1357.97,1621.39,1093.500,2888.14,...,0.0,0.0,0.0,0.0,-3.0,1.0,7123.058318,3.434447e+06,1496.294062,POINT (-4937035.851 1795353.404)
19,-149.990271,57.858883,standard,crater0019,1478.12,48.4531,1311.66,1692.39,764.654,2735.27,...,-1.0,0.0,0.0,0.0,-5.0,1.0,6752.443680,3.140565e+06,1458.545910,POINT (-4548324.050 1754654.492)
20,-145.761170,56.827936,standard,crater0020,2446.51,42.8265,2209.16,2641.62,1880.220,5135.21,...,-1.0,0.0,0.0,0.0,-4.0,1.0,11905.605507,9.882765e+06,2623.930029,POINT (-4420014.135 1723292.110)


In [7]:
gdf_selection.to_file('/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya-less-than-7500m.shp')

In [3]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya-less-than-7500m.shp'

In [9]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 1257/1257 [2:00:12<00:00,  5.74s/it] 


In [4]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 1257/1257 [1:13:36<00:00,  3.51s/it]


## Generate regional (2.0-3.0R) detrended DEM (SLDEM2013, Rayed craters)

In [4]:
scaling_factor = 1.0
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters_detrended/"
craterID = None
overwrite=True

In [6]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=None, overwrite=overwrite)

100%|██████████| 1257/1257 [48:42<00:00,  2.33s/it] 


## Detect rim composite, and update the centre of the crater (Rayed craters)

In [7]:
location_of_craters ='/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya-less-than-7500m.shp'
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/'
suffix = '_Kaguya_SLDEM2013_detrended.tif'
threshold_max_diameter = 7500.0 # no coldspots are larger than 2 km

In [8]:
rim_detection.main(location_of_craters, dem_detrended_folder, shp_folder, suffix, threshold_min=None, threshold_max=threshold_max_diameter, craterID = None)

 27%|██▋       | 334/1257 [21:33<38:28,  2.50s/it]  

crater0434


 33%|███▎      | 409/1257 [27:41<1:15:51,  5.37s/it]

crater0533


 33%|███▎      | 414/1257 [27:54<1:02:16,  4.43s/it]

...

 94%|█████████▍| 1180/1257 [46:03<03:36,  2.81s/it]

crater1390


 98%|█████████▊| 1233/1257 [58:01<05:14, 13.10s/it]

crater1446


100%|██████████| 1257/1257 [1:03:27<00:00,  3.03s/it]


## Update centre of craters
- Merge the centres of craters together
- Merge the polygons of rim composite together (for a faster inspection of results). I think I will here 
- If this step is not working properly for a crater, its id is written down (most likely due to too low resolution of the DEM, or very special conditions)

In [ ]:
out_shapefile = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/final_crater_center/global/rayed_craters_centroids_kaguya-less-than-7500m-manually-checked.shp"
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/polygons'
old_crater_centres = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya-less-than-7500m.shp'

In [ ]:
rim_detection.update_crater_centres(shp_folder, out_shapefile, old_crater_centres, replace_crs=False)

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
To have the right DEMs

In [14]:
location_of_craters = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/final_crater_center/global/rayed_craters_centroids_kaguya-less-than-7500m-manually-checked.shp"
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 6.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/TCOMAP_RayedCraters/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/final_crater_center"

In [10]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance, output_dir, shp_folder, identifier_dem, craterID = None)

100%|██████████| 1256/1256 [1:19:45<00:00,  3.81s/it]


In [11]:
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, output_dir_ortho, shp_folder, identifier_orthoimage, craterID = None)

100%|██████████| 1256/1256 [1:02:31<00:00,  2.99s/it]


In [18]:
scaling_factor = 1.0
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters/"
dem_detrended_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters_detrended/"
craterID = None
overwrite=True
shp_folder = "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/final_crater_center"

In [13]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID=None, overwrite=overwrite)

100%|██████████| 1256/1256 [28:29<00:00,  1.36s/it] 


In [19]:
rim_detection.generated_detrended_dem(location_of_craters, scaling_factor, dem_folder, shp_folder, dem_detrended_folder, craterID='crater0959', overwrite=overwrite)

100%|██████████| 1/1 [00:00<00:00,  1.95it/s]


## Calculate morphometrical parameters from the second ellipse (of the second run)
- load ellipse in pixel coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r
- I could add the possibility to add manually an ellipse for the few cases where the ellipse is not representing well the rim of the crater

In [ ]:
location_of_craters =  "/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/final_crater_center/global/rayed_craters_centroids_kaguya-less-than-7500m-manually-checked.shp"
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/polygons'
dem_folder = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters_detrended/"
suffix = '_Kaguya_SLDEM2013_detrended.tif'
prefix = 'crater'
craterID=None